In [30]:
import os
import glob

In [49]:
script_dir = os.path.dirname(os.path.realpath('dev.ipynb'))

In [70]:
with open(f'{script_dir}/../README.md', 'r') as file:
        readme_lines = file.readlines()

In [71]:
plots = glob.glob(f'{script_dir}/../forecast_plots/*forecast*')
plots = sorted([x for x in plots if 'zoomed' not in x])
zoomed_plots = sorted(glob.glob(f'{script_dir}/../forecast_plots/*zoomed*'))

In [72]:
latest_zoomed = zoomed_plots[-1].split('../')[1]
latest_normal = plots[-1].split('../')[1]

In [73]:
latest_normal

'forecast_plots/20240901_forecast.png'

In [74]:
for i, line in enumerate(readme_lines):
    if '![alt text]' in line:
        if 'zoomed' in line:
            new_line = f'![alt text]({latest_zoomed})\n'
            readme_lines[i] = new_line
        elif 'forecast.png' in line:
            new_line = f'![alt text]({latest_normal})\n'
            readme_lines[i] = new_line
        else:
            continue

In [75]:
readme_lines

["# Forecasting Redfin's Housing Price Index In Los Angeles\n",
 '\n',
 '### Background\n',
 '\n',
 'In 2023, Redfin released their Housing Price Index, which uses the repeat sales pricing method to measure changes in U.S home prices. More information about the index can be found at - https://www.redfin.com/news/redfin-home-price-index/. They release their index monthly, with about a 2 month lag. In this project, I use various time series forecasting methods to forecast their index into the future, allowing interested parties such as real estate professionals, buyers, sellers, and financial institutions to look ahead at what is coming in the housing market. \n',
 '\n',
 'I decided to perform this analysis only on the Los Angeles Area to keep my analysis more focused. \n',
 '\n',
 '![alt text](forecast_plots/original_series.png)\n',
 '\n',
 '### Methods and Data\n',
 '\n',
 'To forecast this time series, I tried using both arima methods and vector autoregression methods. Ultimately, the

In [4]:
readme_lines

["# Forecasting Redfin's Housing Price Index In Los Angeles\n",
 '\n',
 '### Background\n',
 '\n',
 'In 2023, Redfin released their Housing Price Index, which uses the repeat sales pricing method to measure changes in U.S home prices. More information about the index can be found at - https://www.redfin.com/news/redfin-home-price-index/. They release their index monthly, with about a 2 month lag. In this project, I use various time series forecasting methods to forecast their index into the future, allowing interested parties such as real estate professionals, buyers, sellers, and financial institutions to look ahead at what is coming in the housing market. \n',
 '\n',
 'I decided to perform this analysis only on the Los Angeles Area to keep my analysis more focused. \n',
 '\n',
 '![alt text](forecast_plots/original_series.png)\n',
 '\n',
 '### Methods and Data\n',
 '\n',
 'To forecast this time series, I tried using both arima methods and vector autoregression methods. Ultimately, the